In [5]:
! git clone https://github.com/huggingface/transformers.git > /dev/null

Cloning into 'transformers'...
remote: Enumerating objects: 73610, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 73610 (delta 9), reused 2 (delta 0), pack-reused 73584
Receiving objects: 100% (73610/73610), 56.50 MiB | 27.91 MiB/s, done.
Resolving deltas: 100% (52352/52352), done.


In [6]:
! pip install  transformers/

Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 14.1MB/s 
     |████████████████████████████████| 901kB 47.1MB/s 
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2317529 sha256=39dfd53d2c660a630156ed44c8123557ec0f990c1a9a6de2d1854b7586ba5df3
  Stored in directory: /tmp/pip-ephem-wheel-cache-4_kwhmdm/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers


In [7]:
! pip install sentencepiece > /dev/null

In [8]:
! pip install datasets > /dev/null

In [11]:
import transformers

In [12]:
transformers.__version__

'4.7.0.dev0'

In [13]:
tokenizer = transformers.T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
input = tokenizer.encode("明日の天気", return_tensors="pt")
input

tensor([[    9, 14787,    10, 19097,     2]])

/content/transformers/examples/pytorch/language-modeling/run_clm.pyを以下のように編集

- 290行あたりのAutoTokenizerをT5Tokenizerに修正
- from transformers import T5Tokenizer

In [14]:
! nvidia-smi

Fri May 28 14:00:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
!python transformers/examples/pytorch/language-modeling/run_clm.py  \
    --output_dir=finetune_japanese_gpt2 \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --do_train \
    --train_file='/content/drive/MyDrive/other/generate_bz_lyrics/preprocessed.txt' \
    --do_eval \
    --validation_file='/content/drive/MyDrive/other/generate_bz_lyrics/preprocessed.txt' \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --num_train_epochs=50 \
    --overwrite_output_dir

2021-05-28 14:02:17.302173: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
05/28/2021 14:02:18 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
05/28/2021 14:02:18 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=finetune_japanese_gpt2, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=50.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May28_14-02-18_7645963f75d6, logging_strategy=IntervalStrategy.STEPS, logging_first

In [18]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("finetune_japanese_gpt2/checkpoint-4000")

In [20]:
input = tokenizer.encode("夏の日", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=200, num_return_sequences=8)
for pred in tokenizer.batch_decode(output):
  print(pred)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


夏の日</s> 街は呼吸を乱さない 冷たい風 胸はうずきやまない なにもかも捨てられる気がしてた それが一番の罪なんだろうさ 見切り発車も今さら止まれんパラドックス抱いたまんま力尽きるまでやろう 街は呼吸を乱さない 冷たい風 胸はうずきやまない なにもかも捨てられる気がしてた それが一番の罪なんだろうさ 見切発車も今さら止まれんパラドックス抱いたまんま力尽きるまでやろう 街は呼吸を乱さない</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
夏の日</s> 窓を開ければ 風が優しく流れて来て名前も知らない 綺麗な夕陽が 立ちこめられいつまでも手をつないでいられる気がしたんだいつまでも手をつないでいられるわけないどんなに忙しいときでもなぜかこの胸の中だけは昔と変わらないなんにも変わらないいつまでも手をつないでいられるわけないだれかが僕をきらい仲間になるわけないこんなはずじゃないこんな思いを何度もしてるのに何かが変われるはずないそう思ってたふりかざす笑顔が見たいだけのヤツがいつもひとりいる寂しさを飲み込みながら強く手をとりあおう まっすぐな思いを 持つだけなら誰でもなれる希望を 胸に 燃やしながら 君の名を 刻もう</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>

In [25]:
input = tokenizer.encode("明日", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=200, num_return_sequences=8)
for pred in tokenizer.batch_decode(output):
  print(pred)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


明日</s> 午後から天気が崩れるようですね 風も強くなるようです 気をつけてお出かけください<unk> <unk> ああ 今日もうまくいってる 順調な僕のビジネスでもある晩 考えたよ このままどこへ行くのか新しい理想のために上々の現状 捨てたい瞬間がくるだろう いつか<unk> et my way これくらいじゃまだまだget a move on 立ち止まれない<unk> et your way 守りにはいるのは早すぎるlet's breakin' through the wallちらちら スキ見せといて 結局 男をはねかえす誰もが羨望のまなざし そんな女のようさ 夢は無理だからやめときなよって聞き飽きたまわりの声に流されることはもうないよ<unk> et my way これくらいじゃまだまだget a move on 立ち止まれない<unk> et your way 閉ざした君の
明日</s> 朝からなんだかドキドキ...友達にも話したらびっくりされるかもね...悪い予感は的中するかもしれないけど...何も起こらないで欲しい...悪い予感は当たり...悪い予感は的中するかもしれないけど...何も起こらないで欲しい...願い...</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> <

In [28]:
input = tokenizer.encode("まだ暑い", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=200, num_return_sequences=8)
for pred in tokenizer.batch_decode(output):
  print(pred)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


まだ暑い</s> もうすぐおまえは出てゆくんだよ追いかけてばかりいたら しっぺがえし <unk> o! <unk> ad鏡で自分ばっか眺めてため息ばっかついてた若かりし日の美貌は もうどこにも無いよ今を生きるのさ人知れず 日々進化していかなきゃいけないんだから <unk> ey <unk> ey <unk> t's a wonderful day<unk> ey <unk> ey <unk> t's a beautiful day,<unk> ey <unk> ey <unk> t's a wonderful day悲しいのはなぜ?切ないのはなぜなの?あんなこともこんなことも夢じゃないぜいつまでも終わらないで先々どんな出来事あるか知らないがきっと素晴らしい!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
まだ暑い</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s